<center><font size = "10"> Week 9 - Long-term plasticity <center>
<center><font size = "8"> Homework<center>

<font size = "3">Please, write your group number and the names and surnames of the group members here
<br>Group number:
<br>Group member1:
<br>Group member2:
<br>Group member3:
    
__IMPORTANT:__<font size = "3"> Exercises in <font color='red'>red</font> are mandatory assignements to be delivered by next Monday before 23:59.

In [ ]:
import neuron, time
from neuron import h
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
%matplotlib inline

# Recompile external files, load & initialize
!nrnivmodl
neuron.h.load_file("stdrun.hoc");
neuron.h.stdinit();

# <font color= 'red'>Ex 1:</font>  Evolution of synaptic conductance in STDP
    
<font size='3'> Let's explore further the changes of synaptic conductance over time in STDP. Go back to tutorial 1 section 3 and set up everything you need to run it. Once you are ready, expand the plot to 3x2, adding the following in the last row:
   - plot the total excitatory synaptic conductance 𝑔𝐸(𝑡) 
   - plot each of the synaptic conductance changes $g_i$ (similar to the $P_i(t)$ plot)


<font size='3'> With the help of these plots, answer the following questions:
   1) In the above, even though all the presynaptic neurons have the same average firing rate, many of the synapses seem to have been weakened? Did you expect that?
   2) Total synaptic conductance is fluctuating over time. How do you expect 𝑔𝐸 to fluctuate if synapses did not show any STDP like behavior?
   3) Do synaptic weights ever reach a stationary state when synapses show STDP?
    
<font size='3'><font color= 'red'>[2 pts]</font>

In [4]:
# Some functions to help you

# Define parameters of the STDP process
def default_pars_STDP(**kwargs):
    pars = {}
    
    # typical neuron parameters
    pars['V_th'] = -55.     # spike threshold [mV]
    pars['V_reset'] = -75.  # reset potential [mV]
    pars['tau_m'] = 10.     # membrane time constant [ms]
    pars['V_init'] = -65.   # initial potential [mV]
    pars['V_L'] = -75.      # leak reversal potential [mV]
    pars['tref'] = 2.       # refractory time (ms)
    
    # STDP parameters
    pars['A_plus'] = 0.008                   # magnitude of LTP
    pars['A_minus'] = pars['A_plus'] * 1.10  # magnitude of LTD
    pars['tau_stdp'] = 20.                   # STDP time constant [ms]
    
    # simulation parameters
    pars['T'] = 400.  # Total duration of simulation [ms]
    pars['dt'] = .1   # Simulation time step [ms]
    
    # external parameters if any
    for k in kwargs:
        pars[k] = kwargs[k]

    pars['range_t'] = np.arange(0, pars['T'], pars['dt'])  # Vector of discretized time points [ms]
    return pars

def Poisson_generator(pars, rate, n, myseed=False):
    """Generates poisson trains
    Args:
        pars            : parameter dictionary
        rate            : noise amplitute [Hz]
        n               : number of Poisson trains
        myseed          : random seed. int or boolean
    Returns:
        pre_spike_train : spike train matrix, ith row represents whether
                          there is a spike in ith spike train over time
                          (1 if spike, 0 otherwise)
    """
    
    # Retrieve simulation parameters
    dt, range_t = pars['dt'], pars['range_t']
    Lt = range_t.size
    
    # set random seed
    if myseed:
        np.random.seed(seed=myseed)
    else:
        np.random.seed()

    # generate uniformly distributed random variables
    u_rand = np.random.rand(n, Lt)

    # generate Poisson train
    poisson_train = 1. * (u_rand < rate * (dt / 1000.))

    return poisson_train

def generate_P(pars, pre_spike_train_ex):
    """
    track of pre-synaptic spikes
    Args:
        pars               : parameter dictionary
        pre_spike_train_ex : binary spike train input from
                             presynaptic excitatory neuron
    Returns:
        P                  : LTP ratio
    """
    # Get parameters
    A_plus, tau_stdp = pars['A_plus'], pars['tau_stdp']
    dt, range_t = pars['dt'], pars['range_t']
    Lt = range_t.size

    # Initialize
    P = np.zeros(pre_spike_train_ex.shape)
    for it in range(Lt - 1):
        # Calculate the delta increment dP
        dP = -(dt / tau_stdp) * P[:, it] + A_plus * pre_spike_train_ex[:, it + 1]
        # Update P
        P[:, it + 1] = P[:, it] + dP
    return P

# <font color='red'>Ex 2:</font> Evolution of synaptic weights and STDP conditions

<font size='3'> copy and use the widget at the end of tutorial 1 to explore how synaptic weights change over time and given different conditions:
   1) Increase the firing rate (i.e., 30 Hz) of presynaptic neurons, and investigate the effect on the dynamics of synaptic weight distribution
   2) halve the population size of presynaptic neurons, what do you observe? What happens if you double it?
   3) What happens if you double the EPSP time constant? How do you explain this?
   4) And if you decrease it to 4ms? What is the simple, smallest change you can make to reproduce the previous behavior? Can you find another, equivalent change that also does the same? Does this make sense with the previous answers?

<font color= 'red'>[2 pts]</font>

# <font color='red'>Ex 3:</font> Voltage-dependent plasticity model

The clopath et al. model can explain the dependence of not just timing but also frequency on plasticity.

Simulate the model as in tutorial 2 and vary both the spike timing and frequency of stimulation. 
For speed, use just one repetition of 5 spikes and an interstimulus interval of 400.
Use $\Delta t$ values -10, -7, -5, 5, 7, 10 ms and frequency values 2, 5, 10, 20, 40 Hz.
Plot the STDP curves for different frequencies on the same axis (allowing you to compare)

1. What is the effect of frequency on LTD and on LTP?
2. How does this effect result from the model?

<font color='red'> [2 pts]